# Instant Gratification

## Your first Frontier LLM Project!

Let's build a useful LLM solution - in a matter of minutes.

By the end of this course, you will have built an autonomous Agentic AI solution with 7 agents that collaborate to solve a business problem. All in good time! We will start with something smaller...

Our goal is to code a new kind of Web Browser. Give it a URL, and it will respond with a summary. The Reader's Digest of the internet!!

Before starting, you should have completed the setup for [PC](../SETUP-PC.md) or [Mac](../SETUP-mac.md) and you hopefully launched this jupyter lab from within the project root directory, with your environment activated.

## If you're new to Jupyter Lab

Welcome to the wonderful world of Data Science experimentation! Once you've used Jupyter Lab, you'll wonder how you ever lived without it. Simply click in each "cell" with code in it, such as the cell immediately below this text, and hit Shift+Return to execute that cell. As you wish, you can add a cell with the + button in the toolbar, and print values of variables, or try out variations.  

I've written a notebook called [Guide to Jupyter](Guide%20to%20Jupyter.ipynb) to help you get more familiar with Jupyter Labs, including adding Markdown comments, using `!` to run shell commands, and `tqdm` to show progress.

If you prefer to work in IDEs like VSCode or Pycharm, they both work great with these lab notebooks too.  

## If you'd like to brush up your Python

I've added a notebook called [Intermediate Python](Intermediate%20Python.ipynb) to get you up to speed. But you should give it a miss if you already have a good idea what this code does:    
`yield from {book.get("author") for book in books if book.get("author")}`

## I am here to help

If you have any problems at all, please do reach out.  
I'm available through the platform, or at ed@edwarddonner.com, or at https://www.linkedin.com/in/eddonner/ if you'd like to connect (and I love connecting!)

## More troubleshooting

Please see the [troubleshooting](troubleshooting.ipynb) notebook in this folder to diagnose and fix common problems. At the very end of it is a diagnostics script with some useful debug info.

## If this is old hat!

If you're already comfortable with today's material, please hang in there; you can move swiftly through the first few labs - we will get much more in depth as the weeks progress.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Please read - important note</h2>
            <span style="color:#900;">The way I collaborate with you may be different to other courses you've taken. I prefer not to type code while you watch. Rather, I execute Jupyter Labs, like this, and give you an intuition for what's going on. My suggestion is that you do this with me, either at the same time, or (perhaps better) right afterwards. Add print statements to understand what's going on, and then come up with your own variations. If you have a Github account, use this to showcase your variations. Not only is this essential practice, but it demonstrates your skills to others, including perhaps future clients or employers...</span>
        </td>
    </tr>
</table>
<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business value of these exercises</h2>
            <span style="color:#181;">A final thought. While I've designed these notebooks to be educational, I've also tried to make them enjoyable. We'll do fun things like have LLMs tell jokes and argue with each other. But fundamentally, my goal is to teach skills you can apply in business. I'll explain business implications as we go, and it's worth keeping this in mind: as you build experience with models and techniques, think of ways you could put this into action at work today. Please do contact me if you'd like to discuss more or if you have ideas to bounce off me.</span>
        </td>
    </tr>
</table>

In [2]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

# Connecting to OpenAI

The next cell is where we load in the environment variables in your `.env` file and connect to OpenAI.

## Troubleshooting if you have problems:

Head over to the [troubleshooting](troubleshooting.ipynb) notebook in this folder for step by step code to identify the root cause and fix it!

If you make a change, try restarting the "Kernel" (the python process sitting behind this notebook) by Kernel menu >> Restart Kernel and Clear Outputs of All Cells. Then try this notebook again, starting at the top.

Or, contact me! Message me or email ed@edwarddonner.com and we will get this to work.

Any concerns about API costs? See my notes in the README - costs should be minimal, and you can control it at every point. You can also use Ollama as a free alternative, which we discuss during Day 2.

In [3]:
# Load environment variables in a file called .env

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [4]:
openai = OpenAI()

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.
# If it STILL doesn't work (horrors!) then please see the troubleshooting notebook, or try the below line instead:
# openai = OpenAI(api_key="your-key-here-starting-sk-proj-")

# Let's make a quick call to a Frontier model to get started, as a preview!

In [5]:
# To give you a preview -- calling OpenAI with these messages is this easy:

message = "Hello, GPT! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello! Welcome! I'm glad you're here. How can I assist you today?


## OK onwards with our first project

In [6]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [8]:
# Let's try one out. Change the website and add print statements to follow along.

ed = Website("https://platzi.com/")
print(ed.title)
print(ed.text)

Platzi: Plataforma de aprendizaje profesional online
Explorar
Comunidad
Planes
Empresas
Acceder
No hay un limite de edad para aprender. Aprovecha el precio especial
Ahorra:
$991
en Plan expert
Paga a
12
MSI
Termina en:
...
La escuela de tecnología
de Latinoamérica
Más de 5 millones de estudiantes y más de 3.000 empresas aprenden en Platzi
¿Qué quieres aprender?
Empieza cualquier curso sin costo.
O consigue
Platzi para tu empresa
Escuela de
Data Science e Inteligencia Artificial
Escuela de
Ciberseguridad
Escuela de
Liderazgo y Management
Escuela de
English Academy
Escuela de
Desarrollo Web
Escuela de
Marketing Digital
Escuela de
Producto
Escuela de
Contenido Audiovisual
Escuela de
Transformación Digital
Escuela de
Negocios
Escuela de
Talento y Recursos Humanos
Escuela de
Startups
Escuela de
Finanzas e Inversiones
Escuela de
DevOps y Cloud Computing
Escuela de
Diversidad e Inclusión
Escuela de
JavaScript
Escuela de
Programación y Software
Escuela de
Blockchain y Web3
Escuela de
Diseño Gr

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [9]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [10]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [11]:
print(user_prompt_for(ed))

You are looking at a website titled Platzi: Plataforma de aprendizaje profesional online
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Explorar
Comunidad
Planes
Empresas
Acceder
No hay un limite de edad para aprender. Aprovecha el precio especial
Ahorra:
$991
en Plan expert
Paga a
12
MSI
Termina en:
...
La escuela de tecnología
de Latinoamérica
Más de 5 millones de estudiantes y más de 3.000 empresas aprenden en Platzi
¿Qué quieres aprender?
Empieza cualquier curso sin costo.
O consigue
Platzi para tu empresa
Escuela de
Data Science e Inteligencia Artificial
Escuela de
Ciberseguridad
Escuela de
Liderazgo y Management
Escuela de
English Academy
Escuela de
Desarrollo Web
Escuela de
Marketing Digital
Escuela de
Producto
Escuela de
Contenido Audiovisual
Escuela de
Transformación Digital
Escuela de
Negocios
Escuela de
Talento y Recursos Humanos
Escuela de
Startups
Escue

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]

To give you a preview, the next 2 cells make a rather simple call - we won't stretch the might GPT (yet!)

In [12]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [14]:
# To give you a preview -- calling OpenAI with system and user messages:

response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
print(response.choices[0].message.content)
print(response)

Oh, we're starting off with math basics, huh? Well, 2 + 2 is 4. But if you're looking for something more exciting, let me know!
ChatCompletion(id='chatcmpl-Ahl0r99uhufN5NIsF2MUyq5GgvMbQ', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Oh, we're starting off with math basics, huh? Well, 2 + 2 is 4. But if you're looking for something more exciting, let me know!", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1734992185, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_0aa8d3e20b', usage=CompletionUsage(completion_tokens=36, prompt_tokens=26, total_tokens=62, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


## And now let's build useful messages for GPT-4o-mini, using a function

In [15]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [16]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Platzi: Plataforma de aprendizaje profesional online\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nExplorar\nComunidad\nPlanes\nEmpresas\nAcceder\nNo hay un limite de edad para aprender. Aprovecha el precio especial\nAhorra:\n$991\nen Plan expert\nPaga a\n12\nMSI\nTermina en:\n...\nLa escuela de tecnología\nde Latinoamérica\nMás de 5 millones de estudiantes y más de 3.000 empresas aprenden en Platzi\n¿Qué quieres aprender?\nEmpieza cualquier curso sin costo.\nO consigue\nPlatzi para tu empresa\nEscuela de\nData Science e Inteligencia Artificial\nEscuela de\nCiberseguridad\nEscuela de\nLiderazgo y Manag

## Time to bring it together - the API for OpenAI is very simple!

In [17]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [18]:
summarize("https://platzi.com/")

'# Platzi: Plataforma de Aprendizaje Profesional Online\n\nPlatzi es una plataforma de aprendizaje en línea enfocada en la educación tecnológica en América Latina, con más de 5 millones de estudiantes y más de 3,000 empresas que utilizan sus servicios. Ofrece una variedad de cursos en diversas disciplinas, tales como:\n\n- Data Science e Inteligencia Artificial\n- Ciberseguridad\n- Liderazgo y Management\n- Desarrollo Web\n- Marketing Digital\n- Blockchain y Web3\n- Entre otros\n\nLos estudiantes pueden comenzar cualquier curso sin costo y tienen la opción de generar rutas de aprendizaje personalizadas que los guían desde el nivel principiante hasta el profesional. La plataforma también incluye características como pruebas interactivas, acceso offline a las clases y certificaciones oficiales en colaboración con entidades como ETS para el TOEFL y CompTIA para seguridad digital.\n\nAdemás, Platzi ofrece planes especiales y oportunidades para empresas en la formación de sus equipos, así c

In [19]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [20]:
display_summary("https://platzi.com")

# Platzi: Plataforma de Aprendizaje Profesional Online

Platzi es una plataforma de educación en línea enfocada en el desarrollo profesional, especialmente en tecnología, con más de 5 millones de estudiantes y más de 3,000 empresas utilizando sus servicios. La plataforma ofrece una amplia gama de cursos y programas en diversas áreas como Ciencia de Datos, Ciberseguridad, Marketing Digital, Desarrollo Web, y más.

## Ofertas y Características

- **Planes de Aprendizaje**: Incluye un precio especial para el Plan Expert y opciones de pago a 12 meses sin interés.
- **Metodologías de Aprendizaje**: Rutas profesionales que guían a los estudiantes desde nivel principiante hasta profesional, con la posibilidad de aprender de forma interactiva.
- **Certificaciones**: Alianzas para certificaciones oficiales, incluyendo TOEFL y herramientas de tecnología cloud.
- **Acceso a Cursos**: Se pueden empezar cursos sin costo y descargar clases para aprender sin conexión a Internet.
- **Soporte Educativo**: La plataforma cuenta con inteligencia artificial y una comunidad para resolver dudas.

## Anuncios

- **Promociones**: Un llamado a aprovechar ofertas especiales, destacando el ahorro en el Plan Expert.

Platzi se posiciona como un líder educativo en América Latina, con un fuerte enfoque en la formación continua y el desarrollo de habilidades técnicas para el mercado laboral.

# Let's try more websites

Note that this will only work on websites that can be scraped using this simplistic approach.

Websites that are rendered with Javascript, like React apps, won't show up. See the community-contributions folder for a Selenium implementation that gets around this. You'll need to read up on installing Selenium (ask ChatGPT!)

Also Websites protected with CloudFront (and similar) may give 403 errors - many thanks Andy J for pointing this out.

But many websites will work just fine!

In [21]:
display_summary("https://cnn.com")

# Summary of CNN Website

CNN's website provides up-to-date news articles, videos, and live broadcasts covering various topics including U.S. and world news, politics, business, health, entertainment, climate, science, and sports. It features specific sections dedicated to ongoing global conflicts such as the Ukraine-Russia war and the Israel-Hamas war.

## Recent News Highlights:
- **Matt Gaetz**: An ethics panel found evidence against Matt Gaetz for violating Florida state laws related to sexual misconduct. 
- **Trump and Politics**: Trump has made bold statements concerning immigrants and has expressed concerns about Elon Musk's influence in the GOP. His tensions with political dynamics are also covered.
- **International Affairs**: Insights into the siege faced by Venezuelan opposition members and reports on North Korean soldiers using fake Russian IDs.
- **Nissan-Honda Merger**: The two automakers announced plans to merge, creating the world’s third-largest automotive company.
- **Cultural Updates**: King Charles to deliver a Christmas message in a break from tradition, and significant renovations in historical sites like Rome’s Trevi Fountain.

The site encourages user interaction by soliciting feedback on advertisements and offers a variety of multimedia content including articles, videos, podcasts, and live TV options.

In [22]:
display_summary("https://anthropic.com")

# Anthropic Overview

Anthropic is an AI safety and research company based in San Francisco, focused on developing reliable and beneficial AI systems. Their interdisciplinary team brings expertise in machine learning, physics, policy, and product development.

## Key Offerings

- **Claude 3.5 Sonnet**: The most advanced AI model from Anthropic, now available for users.
- **Claude API**: Enables developers to build AI-powered applications.
- **Claude for Enterprise**: Tailored solutions for organizations looking to integrate AI technology.

## Recent Announcements

- **October 22, 2024**: Introduction of new features including the Claude 3.5 Sonnet and Claude 3.5 Haiku, along with the rollout of "computer use".
- **September 4, 2024**: Updates regarding Claude for Enterprise.

## Research Focus

Anthropic emphasizes AI safety and has published research on topics such as:
- **Constitutional AI**: An approach aimed at ensuring harmlessness in AI systems.
- **Core Views on AI Safety**: A detailed exploration of the principles guiding their AI safety efforts. 

Overall, Anthropic is committed to advancing AI technology while prioritizing safety and ethical considerations.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise, you experienced calling the Cloud API of a Frontier Model (a leading model at the frontier of AI) for the first time. We will be using APIs like OpenAI at many stages in the course, in addition to building our own LLMs.

More specifically, we've applied this to Summarization - a classic Gen AI use case to make a summary. This can be applied to any business vertical - summarizing the news, summarizing financial performance, summarizing a resume in a cover letter - the applications are limitless. Consider how you could apply Summarization in your business, and try prototyping a solution.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you continue - now try yourself</h2>
            <span style="color:#900;">Use the cell below to make your own simple commercial example. Stick with the summarization use case for now. Here's an idea: write something that will take the contents of an email, and will suggest an appropriate short subject line for the email. That's the kind of feature that might be built into a commercial email tool.</span>
        </td>
    </tr>
</table>

In [ ]:
# Step 1: Create your prompts

system_prompt = "something here"
    Can be pasted here
"""

# Step 2: Make the messages list

messages = [] # fill this in

# Step 3: Call OpenAI

response =

# Step 4: print the result

print(

## An extra exercise for those who enjoy web scraping

You may notice that if you try `display_summary("https://openai.com")` - it doesn't work! That's because OpenAI has a fancy website that uses Javascript. There are many ways around this that some of you might be familiar with. For example, Selenium is a hugely popular framework that runs a browser behind the scenes, renders the page, and allows you to query it. If you have experience with Selenium, Playwright or similar, then feel free to improve the Website class to use them. In the community-contributions folder, you'll find an example Selenium solution from a student (thank you!)

# Sharing your code

I'd love it if you share your code afterwards so I can share it with others! You'll notice that some students have already made changes (including a Selenium implementation) which you will find in the community-contributions folder. If you'd like add your changes to that folder, submit a Pull Request with your new versions in that folder and I'll merge your changes.

If you're not an expert with git (and I am not!) then GPT has given some nice instructions on how to submit a Pull Request. It's a bit of an involved process, but once you've done it once it's pretty clear. As a pro-tip: it's best if you clear the outputs of your Jupyter notebooks (Edit >> Clean outputs of all cells, and then Save) for clean notebooks.

PR instructions courtesy of an AI friend: https://chatgpt.com/share/670145d5-e8a8-8012-8f93-39ee4e248b4c